<a href="https://colab.research.google.com/github/alfrizzle/NLP-Projects/blob/master/sagemaker_train/sagemaker_train_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# About

This notebook is an implementation of the model training code in the `sagemaker_train` directory, which contains the following folders and scripts

*   `src` (directory)
  * `__init__.py`
  * `model.py`: entry point used to train the model; uses input arguments
      * includes `ClassifierDataset` class that converts a `csv` or `json` file into a torch dataset, which is used as input data for the model training step
      * `def main` is the main code used to train and evaluate the model using argparse
      * `def preprocess_data.py` splits the training dataset into train/test (default test size is 0.2)
  * `utils.py`: converts an object (`csv` or `json` file) into a Pandas dataframe, which is used in `model.py` as input data
*   `data` (directory)
  * includes training data files that can be used for the model
  * recommend using `wiki_attacks.csv` to train a decently-performing model
* `model` (directory): where the trained model will be saved
* `eval_results` (directory): where the evaluation results (`json) format) will be saved

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Prepare Environment

If running the code in notebook, be sure to install the following libraries

In [2]:
!pip install torch # version 1.10.0
!pip install transformers # version 4.15.0

Make sure the working directory is set to the `src` directory

In [3]:
import os

os.chdir('/content/drive/MyDrive/sm_training/sagemaker_train/src')
!pwd

/content/drive/MyDrive/sm_training/sagemaker_train/src


# Defining Training Arguments & Hyperparameters

The following parameters can be specified and passed into training script:

```
    # Hyperparameters from launch_training_job.py get passed in as command line args.
    parser.add_argument('--input_path', type=str)
    parser.add_argument('--train_size', type=float, default=.85)
    parser.add_argument('--adam_epsilon', type=float, default=1e-8)
    parser.add_argument('--epochs', type=int, default=2)
    parser.add_argument('--learning_rate', type=float, default=5e-5)
    parser.add_argument('--weight_decay', type=float, default=0.0)
    parser.add_argument('--max_data_rows', type=int, default=None)
    parser.add_argument('--max_sequence_length', type=int, default=128)
    parser.add_argument('--model_name', type=str, default='distilbert-base-uncased')
    parser.add_argument('--train_batch_size', type=int, default=16)
    parser.add_argument('--valid_batch_size', type=int, default=128)
    parser.add_argument('--file_type', type=str, default='csv') # specify whether input file is csv or json (has to be one of the two)
    parser.add_argument('--eval_dir', type=str, default='../eval_results') # set this to SM's model_dir path when using in SageMaker
    parser.add_argument('--model_dir', type=str, default='../model') # where trained model is saved when running the script locally (outside of SageMaker)
```

Training Arguments in the `model.py` script includes additional parameters, such as `warmup_steps` (default value of 500) and `logging_steps` (default value of 10). Be sure to change these values if you want to decrease/increase the logging frequency.

```
    training_args = TrainingArguments(
        output_dir=os.path.join(args.model_dir, "output"),
        num_train_epochs=args.epochs,
        per_device_train_batch_size=args.train_batch_size,
        per_device_eval_batch_size=args.valid_batch_size,
        learning_rate=args.learning_rate,
        adam_epsilon=args.adam_epsilon,
        warmup_steps=500,
        weight_decay=args.weight_decay,
        logging_dir=os.path.join(args.model_dir, "logs"),
        logging_steps=10,
        evaluation_strategy="steps",
        load_best_model_at_end=True
    )
```

These are sample argument values
```
# model_name = 'distilbert-base-uncased'
model_name = 'distilroberta-base'
max_sequence_length = 128
input_file = '/content/drive/MyDrive/datasets/wiki_attacks.csv'
output_dir = 'results'
epochs = 2
train_batch_size = 32
valid_batch_size = 128
learning_rate = 5e-5
adam_epsilon = 1e-8
weight_decay = 0.0
logging_dir = 'logs'
```

# Begin Training Job

Run the following command to start the training in notebook

In [15]:
!python trainer.py --model_name distilroberta-base --input_path /content/drive/MyDrive/sm_training/sagemaker_train/data/wiki_attacks_sample.csv --file_type csv --test_size .20 --epochs 45 --train_batch_size 16 --model_dir /content/drive/MyDrive/sm_training/sagemaker_train/test_model --warmup_steps 100 --logging_steps 10 --eval_dir /content/drive/MyDrive/sm_training/sagemaker_train/test_model/evaluation

2022-01-09 06:54:20,433 - __main__ - INFO -  Dataset contains 1000 rows
Data contains 1000 rows
2022-01-09 06:54:23,930 - __main__ - INFO -  loaded train_dataset length is: 800
2022-01-09 06:54:23,930 - __main__ - INFO -  loaded test_dataset length is: 200
{'input_ids': tensor([    0,  1437,  1437, 45994, 32588, 43292,    42,   177,  1326,    98,
        21905,   179,  4045,   939,  1034,   127,  4252, 13079,    24,    13,
          162,     2,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1

## Save Model Artifact as model.tar.gz

In [ ]:
model_dir = '/content/drive/MyDrive/sm_training/sagemaker_train/test_model'

In [ ]:
import tarfile
import os.path

def make_tarfile(output_filename, source_dir):
    with tarfile.open(output_filename, "w:gz") as tar:
        tar.add(source_dir, arcname=os.path.basename(source_dir))

In [ ]:
!pwd

/content/drive/My Drive/sm_training/sagemaker_train/src


In [ ]:
make_tarfile('wiki_attacks_model.tar.gz', model_dir)

# Inference Code (in progress)

Resources:
* https://github.com/aws-samples/amazon-sagemaker-bert-pytorch/blob/master/code/deploy_ei.py
* https://github.com/aws/sagemaker-pytorch-inference-toolkit/blob/master/src/sagemaker_pytorch_serving_container/default_pytorch_inference_handler.py
* https://github.com/aws/sagemaker-inference-toolkit/blob/master/src/sagemaker_inference/default_inference_handler.py

Import dependencies

Note: working directory should still be the `src` directory

In [ ]:
import argparse
import os
import json

# import boto3
import numpy as np
import pandas as pd
from sklearn.metrics import precision_score, recall_score, average_precision_score, roc_auc_score, f1_score
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler
from transformers import AdamW, AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments

from utils import read_object

## Custom Model Inference Script

This code is based on the SageMaker hugginface inference toolkit's handler_service.py

https://github.com/aws/sagemaker-huggingface-inference-toolkit/blob/main/src/sagemaker_huggingface_inference_toolkit/handler_service.py

In [ ]:
import importlib
import logging
import os
import sys
import time
from abc import ABC

from sagemaker_inference import content_types, environment, utils
from transformers.pipelines import SUPPORTED_TASKS

from mms.service import PredictionException
from sagemaker_huggingface_inference_toolkit import decoder_encoder
from sagemaker_huggingface_inference_toolkit.transformers_utils import (
    _is_gpu_available,
    get_pipeline,
    infer_task_from_model_architecture,
)

In [ ]:
def load(self, model_dir):
  """
  The Load handler is responsible for loading the Hugging Face transformer model.
  It can be overridden to load the model from storage
  Returns:
  hf_pipeline (Pipeline): A Hugging Face Transformer pipeline.
  """
  # gets pipeline from task tag
  if "HF_TASK" in os.environ:
    hf_pipeline = get_pipeline(task=os.environ["HF_TASK"], model_dir=model_dir, device=self.device)
  elif "config.json" in os.listdir(model_dir):
    task = infer_task_from_model_architecture(f"{model_dir}/config.json")
    hf_pipeline = get_pipeline(task=task, model_dir=model_dir, device=self.device)
  else:
    raise ValueError(
        f"You need to define one of the following {list(SUPPORTED_TASKS.keys())} as env 'TASK'.", 403
        )
  return hf_pipeline

In [ ]:
def postprocess(self, prediction, accept):
  """
  The postprocess handler is responsible for serializing the prediction result to
  the desired accept type, can handle JSON.
  The postprocess handler can be overridden for inference response transformation
  Args:
    prediction (dict): a prediction result from predict
    accept (str): type which the output data needs to be serialized
  Returns: output data serialized
  """
  return decoder_encoder.encode(prediction, accept)

### ***** Use the Code Below *****

In [ ]:
import os 
import json 
import torch
from sagemaker_huggingface_inference_toolkit import decoder_encoder
from transformers import AutoModelForSequenceClassification, AutoTokenizer

def model_fn(model_dir):
    """
    Load the model and tokenizer for inference 
    """
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    tokenizer = AutoTokenizer.from_pretrained(model_dir)
    model = AutoModelForSequenceClassification.from_pretrained(model_dir).to(device)
    
    model_dict = {'model':model, 'tokenizer':tokenizer}
    
    return model_dict 


def input_fn(request_body, request_content_type):
    """
    Transform the input request to a dictionary
    """
    assert request_content_type=='application/json'
    
    request = json.loads(request_body)
    
    # decoded_input_data = decoder_encoder.decode(request_body, request_content_type)
    
    # return decoded_input_data

    return request


def predict_fn(input_data, model):
    """
    Make a prediction with the model
    """ 

    text = input_data.pop('inputs')
    parameters = input_data.pop('parameters', None)
    
    tokenizer = model['tokenizer']
    model = model['model']

    # Parameters may or may not be passed    
    input_ids = tokenizer(text, truncation=True, padding='longest', return_tensors="pt").input_ids
    # output = model.generate(input_ids, **parameters) if parameters is not None else model.generate(input_ids)
    
    # pass inputs with all kwargs in data
    if parameters is not None:
      prediction = model(input_ids, **parameters)
    else:
      prediction = model(input_ids)
    
    return prediction
    
    # return tokenizer.batch_decode(output, skip_special_tokens=True)[0]


def output_fn(prediction, content_type):
    """
    Return model's prediction
    """
    assert content_type == 'application/json'
    return decoder_encoder.encode(prediction, accept)

Set variables for inference code

In [ ]:
model_dir = '/content/drive/MyDrive/sm_training/sagemaker_train/test_model'

In [ ]:
def model_fn(model_dir):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("================ objects in model_dir ===================")
    print(os.listdir(model_dir))
    model = AutoModelForSequenceClassification.from_pretrained(model_dir)
    print("================ model loaded ===========================")
    return model.to(device)

In [ ]:
model_fn(model_dir)

In [ ]:
def input_fn(request_body, request_content_type):
    """An input_fn that loads a pickled tensor"""
    if request_content_type == "application/json":
        data = json.loads(request_body)
        print("================ input text ===============")
        print(data)
        
        if isinstance(data, str):
            data = [data]
        elif isinstance(data, list) and len(data) > 0 and isinstance(data[0], str):
            pass
        else:
            raise ValueError("Unsupported input type. Input type can be a string or an non-empty list. \
                             I got {}".format(data))
                       
        #encoded = [tokenizer.encode(x, add_special_tokens=True) for x in data]
        #encoded = tokenizer(data, add_special_tokens=True) 
        
        # for backward compatibility use the following way to encode 
        # https://github.com/huggingface/transformers/issues/5580
        input_ids = [tokenizer.encode(x, add_special_tokens=True) for x in data]
        
        print("================ encoded sentences ==============")
        print(input_ids)

        # pad shorter sentence
        padded =  torch.zeros(len(input_ids), MAX_LEN) 
        for i, p in enumerate(input_ids):
            padded[i, :len(p)] = torch.tensor(p)
     
        # create mask
        mask = (padded != 0)
        
        print("================= padded input and attention mask ================")
        print(padded, '\n', mask)

        return padded.long(), mask.long()
    raise ValueError("Unsupported content type: {}".format(request_content_type))
    

def predict_fn(input_data, model):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    input_id, input_mask = input_data
    input_id = input_id.to(device)
    input_mask = input_mask.to(device)
    print("============== encoded data =================")
    print(input_id, input_mask)
    with torch.no_grad():
        y = model(input_id, attention_mask=input_mask)[0]
        print("=============== inference result =================")
        print(y)
    return y

## Renaming dict objects

In [9]:
eval_result = {'epoch': 2.0,
 'eval_f1_score': 0.7644970414201182,
 'eval_loss': 0.14108391106128693,
 'eval_precision': 0.7916666666666666,
 'eval_recall': 0.7391304347826086,
 'eval_roc_auc': 0.8576370669562777,
 'eval_runtime': 43.9869,
 'eval_samples_per_second': 90.936,
 'eval_steps_per_second': 0.727}

In [10]:
eval_result

{'epoch': 2.0,
 'eval_f1_score': 0.7644970414201182,
 'eval_loss': 0.14108391106128693,
 'eval_precision': 0.7916666666666666,
 'eval_recall': 0.7391304347826086,
 'eval_roc_auc': 0.8576370669562777,
 'eval_runtime': 43.9869,
 'eval_samples_per_second': 90.936,
 'eval_steps_per_second': 0.727}

In [11]:
# Export eval results as json
# keep = ['eval_loss', 'eval_precision', 'eval_recall', 'eval_f1_score', 'eval_roc_auc']
keep = ['eval_f1_score', 'eval_loss', 'eval_precision', 'eval_recall', 'eval_roc_auc']
partial_dict = {k: v for k, v in eval_result.items() if k in keep}
partial_dict

{'eval_f1_score': 0.7644970414201182,
 'eval_loss': 0.14108391106128693,
 'eval_precision': 0.7916666666666666,
 'eval_recall': 0.7391304347826086,
 'eval_roc_auc': 0.8576370669562777}

In [43]:
# # inititialising dictionary
# ini_dict = {'nikhil': 1, 'vashu' : 5, 
#             'manjeet' : 10, 'akshat' : 15}
  
# initializing new dict key names
new_keys = ['F1', 'Loss', 'Precision', 'Recall', 'ROC_AUC']
# new_names = ['Loss', 'Precision', 'Recall', 'F1', 'ROC_AUC']
  
# # printing initial json
# print ("initial 1st dictionary", partial_dict)

# print('\n')
  
# changing keys of dictionary
final_dict = dict(zip(new_keys, list(partial_dict.values())))
  
# printing final result
# print ("final dictionary", str(final_dict))
final_dict

{'F1': 0.7644970414201182,
 'Loss': 0.14108391106128693,
 'Precision': 0.7916666666666666,
 'ROC_AUC': 0.8576370669562777,
 'Recall': 0.7391304347826086}

In [12]:
dict_key_map = {'eval_loss'      : 'Loss' ,
                'eval_precision' : 'Precision' ,
                'eval_recall'    : 'Recall', 
                'eval_f1_score'  : 'f1',
                'eval_roc_auc'   : 'ROC_AUC'
                }

In [14]:
# target_dict = {'k1':'v1', 'k2':'v2', 'k3':'v3'}
new_keys = ['F1','Loss','Precision', 'Recall', 'ROC_AUC']

for key,n_key in zip(partial_dict.keys(), new_keys):
    partial_dict[n_key] = partial_dict.pop(key)

partial_dict

{'Loss': 0.7916666666666666,
 'Precision': 0.7391304347826086,
 'ROC_AUC': 0.7644970414201182,
 'Recall': 0.8576370669562777,
 'eval_loss': 0.14108391106128693}

In [13]:
final_dict = {names_key.get(k, k): v for k, v in partial_dict.items()}

In [14]:
final_dict

{'Loss': 0.14108391106128693,
 'Precision': 0.7916666666666666,
 'ROC_AUC': 0.8576370669562777,
 'Recall': 0.7391304347826086,
 'f1': 0.7644970414201182}

## Reading Log Files with Tensorboard

In [23]:
import pandas as pd

def parse_run_progresscsv(run_folder,
                          fillna=False,
                          verbose=False) -> pd.DataFrame:
  """Create a pandas DataFrame object from progress.csv per convention."""
  # Try progress.csv or log.csv from folder
  detected_csv = None
  for fname in ('progress.csv', 'log.csv'):
    p = os.path.join(run_folder, fname)
    if exists(p):
      detected_csv = p
      break

  # maybe a direct file path is given instead of directory
  if detected_csv is None:
    if exists(run_folder) and not isdir(run_folder):
      detected_csv = run_folder

  if detected_csv is None:
    raise FileNotFoundError(os.path.join(run_folder, "*.csv"))

  # Read the detected file `p`
  if verbose:
    print(f"parse_run (csv): Reading {detected_csv}",
          file=sys.stderr, flush=True)  # yapf: disable

  with open(detected_csv, mode='r') as f:
    df = pd.read_csv(f)

  if fillna:
    df = df.fillna(0)

  return df


def parse_run_tensorboard(run_folder,
                          fillna=False,
                          verbose=False) -> pd.DataFrame:
  """Create a pandas DataFrame from tensorboard eventfile or run directory."""
  event_file = list(
      sorted(glob(os.path.join(run_folder, '*events.out.tfevents.*'))))

  if not event_file:  # no event file detected
    raise pd.errors.EmptyDataError(f"No event file detected in {run_folder}")
  event_file = event_file[-1]  # pick the last one
  if verbose:
    print(f"parse_run (tfevents) : Reading {event_file} ...",
          file=sys.stderr, flush=True)  # yapf: disable

  from collections import defaultdict

  from tensorflow.core.util import event_pb2
  from tensorflow.python.framework.dtypes import DType
  try:
    # avoid DeprecationWarning on tf_record_iterator
    from tensorflow.python._pywrap_record_io import RecordIterator

    def summary_iterator(path):
      for r in RecordIterator(path, ""):
        yield event_pb2.Event.FromString(r)
  except:
    from tensorflow.python.summary.summary_iterator import \
        summary_iterator  # type: ignore